<a href="https://colab.research.google.com/github/sachinmkotian/CloudxLabs_sachin_2024/blob/main/RAG_using_Langchain_and_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

import os


In [72]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['OPENAI_API_KEY'] = "******"
os.environ['LANGCHAIN_API_KEY'] = "******"
os.environ["LANGCHAIN_PROJECT"] = "default"

In [73]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [74]:
#Indexing existing documents
#Indexing : Fish out documents similar to my question...
#Document with similar words are in similar space...this is the fundamental idea for Indexing
#Questions are then compared to the documents based on numerical comparisions

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        #Use BeautifulSoup library to parse objects
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    ),
)

#creates dict type objects
blog_docs = loader.load()

#Split
# Use RecursiveCharacterTextSplitter Recursively tries to split by different characters to find one that works
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300,chunk_overlap=50)
splits = text_splitter.split_documents(blog_docs)

#Retrieval
#Convert documents to numerical representations which can be searched aka Embedding
#Documents are split and compressed to a Vector. Vectors are indexed
#Store the documents in a Chroma vector using OpenAI Embeddings
vectorstore = Chroma.from_documents(documents=splits,embedding=OpenAIEmbeddings())

#vector store retriever is a retriever that uses a vector store to retrieve documents. It is a lightweight wrapper around the vector store class to make it conform to the retriever interface.
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
docs = retriever.get_relevant_documents("What is task decomposition?")
len(docs)

1

In [75]:

#Generation

from langchain.prompts import ChatPromptTemplate

#A prompt template is a prompt that includes replaceable variables.
#Prompt templates enable you to test how different prompt formats perform with different prompt data, without requiring you to write multiple individual prompts.

template = """Answer the question based only on the following context:
{context}

Question:{question}
"""
prompt = ChatPromptTemplate.from_template(template)
prompt

#LLM

llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)

#langchain has a expression language
chain = prompt | llm

chain.invoke({"context":docs,"question":"What is task decomposition?"})

from langchain import hub

#Pull Langchain prompt object which has human text.
prompt_hub_rag = hub.pull("rlm/rag-prompt")
prompt_hub_rag


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [76]:
rag_chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#StrOuputParser is a OutputParser that parses LLMResult into the top likely string
rag_chain.invoke("Can you give a Agent System Overview in 5 points?")

'1. Utilizes internet access for searches and information gathering.\n2. Manages long term memory effectively.\n3. Employs GPT-3.5 powered Agents for delegation of simple tasks.\n4. Outputs files as needed.\n5. Evaluates performance by continuously reviewing actions, self-criticizing behavior, reflecting on past decisions, and aiming for efficiency in task completion.'